In [3]:
import os
import cv2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Read meta data 

In [4]:
people_df = pd.read_csv('C:/Users/galin/Downloads/LFW dataset/people.csv')
people_train_df = pd.read_csv('C:/Users/galin/Downloads/LFW dataset/peopleDevTrain.csv')
people_test_df = pd.read_csv('C:/Users/galin/Downloads/LFW dataset/peopleDevTest.csv')

In [5]:
people_train_df

,name,images
0,AJ_Cook,1
1,Aaron_Eckhart,1
2,Aaron_Patterson,1
3,Aaron_Peirsol,4
4,Aaron_Pena,1
...,...,...
4033,Zinedine_Zidane,6
4034,Ziwang_Xu,1
4035,Zorica_Radovic,1
4036,Zumrati_Juma,1
